# Gemini Token


In [ ]:
%pip install --upgrade --user google-cloud-aiplatform
%pip install sentencepiece

In [18]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [19]:
PROJECT_ID = os.environ['PROJECT_ID']
LOCATION = os.environ['LOCATION']
BUCKET = os.environ['BUCKET']

In [20]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [74]:
from vertexai.generative_models import (
    GenerativeModel,
    Part,
    Image,  
)

In [75]:
import vertexai.preview
from vertexai.preview.tokenization import get_tokenizer_for_model

def print_token_tokenizer(model_name,prompt=None):

    # init local tokenzier
    tokenizer = get_tokenizer_for_model(model_name)

    # Count Toke
    response = tokenizer.count_tokens(prompt)
    print(f"Tokens count: {response.total_tokens}")

    # Compute Tokens
    response = tokenizer.compute_tokens(prompt)
    print(f"Tokens list: {response.tokens_info}")

In [23]:
def print_token(prompt, image_prompt=None, model_name="gemini-1.5-flash-002", ):
    model = GenerativeModel(model_name)
    if image_prompt is not None:
        org_image = Part.from_uri(f"{BUCKET}/{image_prompt}",mime_type="image/png")
        response = model.count_tokens([prompt,org_image])
    else:
        response = model.count_tokens(prompt)
    # Prompt tokens count
    tokens = response.total_tokens
    chars = response.total_billable_characters
    print(f"Prompt Token Count: {tokens}")
    print(f"Prompt Character Count: {chars}")

    # Send text to Gemini
    #response = model.generate_content(prompt)

    # Response tokens count
    #usage_metadata = response.usage_metadata
    #print(f"Prompt Token Count: {usage_metadata.prompt_token_count}")
    #print(f"Candidates Token Count: {usage_metadata.candidates_token_count}")
    #print(f"Total Token Count: {usage_metadata.total_token_count}")
    return tokens, chars


In [77]:
def print_token_image(image_prompt, model_name="gemini-1.5-flash-002"):
    model = GenerativeModel(model_name)
    
    #org_image = Part.from_uri(f"{BUCKET}/{image_prompt}",mime_type="image/png")

    input_image = Part.from_image(Image.load_from_file(image_prompt))
    response = model.count_tokens(input_image)

    print(response)
    # Prompt tokens count
    tokens = response.total_tokens
    chars = response.total_billable_characters
    print(f"Prompt Token Count: {tokens}")
    print(f"Prompt Character Count: {chars}")

    return tokens, chars   

In [57]:
import random
from PIL import Image as PIL_Image

def create_random_image(width, height):
    output_path = f"./images/random/output_{width}_{height}.png"
    """
    Creates an image with random RGB pixel values and saves it as a PNG file.

    Args:
        width (int): The width of the image in pixels.
        height (int): The height of the image in pixels.
        output_path (str): The path to save the image.
    """

    # Create a new image with a white background
    img = PIL_Image.new('RGB', (width, height), color='white')

    # Generate random pixel values
    pixels = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
              for _ in range(width * height)]

    # Put the random pixels into the image
    img.putdata(pixels)

    # Save the image
    img.save(output_path)
    return output_path

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=["width", "height", "w x h pixels", "tokens"])

image_sizes = [[200,200],[384,384], [768,768], [1024,1024], [768,1408], [1408, 768], [896, 1280], [1280, 896], [1536, 1566], [1152,2016], [2016, 1134], [1344, 1792], [1792, 1344], [2048, 2048], [2816,1536], [4096,4096]]
#image_sizes = [ [4096,4096]]

for width, height in image_sizes:
    print(f"Image width: {width}")
    pixels= width * height
    tokens, chars = print_token_image(create_random_image(width, height))
    df = df._append({'width': width, 'height': height,'w x h pixels': pixels, 'tokens':tokens }, ignore_index=True)

In [71]:
df_sorted = df.sort_values(by='w x h pixels', ascending=True)
display(df_sorted)

,width,height,w x h pixels,tokens
0,200,200,40000,258
1,384,384,147456,258
2,768,768,589824,258
3,1024,1024,1048576,258
4,768,1408,1081344,258
5,1408,768,1081344,258
6,896,1280,1146880,258
7,1280,896,1146880,258
10,2016,1134,2286144,258
9,1152,2016,2322432,258


In [33]:
def print_price(tokens, chars):
    print(f"Flash 1.5      price {(chars/1000)*0.00001875:0.10f}")
    print(f"Flash 2.0      price {(tokens/1000000)*0.15:0.10f}")
    print(f"Flash 2.0 Lite price {(tokens/1000000)*0.075:0.10f}")

In [17]:
prompt ="""
다음 뉴스를 요약해줘:
이번 주에는 기온이 오름세를 보이며 한파의 기세가 점차 누그러지겠습니다.
오늘 아침도 경기 내륙과 강원 내륙 산지, 충청 내륙, 전북 동부, 경북 내륙 지역은 한파특보가 내려진 가운데 춥긴 추웠는데요.
오늘 아침 기온은 서울이 영하 7.5도 등 전국 대부분 지역이 어제보다는 1도에서 7도 정도 높았고, 한낮 기온도 전국이 영상으로 올라서 어제보다 1도에서 3도 정도 높겠습니다.
내일 아침에는 서울이 영하 5도로 오늘보다 3도 정도 높겠고, 한낮에는 영상 6도까지 올라가 평년 수준을 웃돌겠습니다.
지금도 대설주의보가 내려진 전북 고창과 부안, 군산에는 눈이 오는 곳이 있습니다.
오늘은 오전까지 충남 서해안과 전북 서해안에 1에서 5센티미터, 경기 남부와 충청 내륙, 전북 내륙과 광주, 전남 서부 지역에 1센티미터 안팎의 눈이 올 것으로 보입니다.
반면 눈이 내리지 않은 동해안과 경남 동부 지역은 건조특보가 계속되겠습니다.
특히 오늘은 해안 지방에 바람이 순간 풍속 초속 15미터 이상으로 강하게 부는 곳이 있어 불씨 관리에 신경 쓰셔야겠습니다.
낮 기온은 서울과 춘천이 2도, 광주가 4도, 대전, 대구 5도로 어제보다 조금 높겠습니다.
바다의 물결은 동해 먼바다에서 최고 3.5미터로 높게 일겠습니다.
정월대보름인 모레 수요일에는 전국에 눈이나 비가 내리겠습니다.
날씨 전해드렸습니다.
"""

In [37]:
tokens, chars =print_token(prompt)
print_price(tokens, chars)

Prompt Token Count: 495
Prompt Character Count: 518
Flash 1.5      price 0.0000097125
Flash 2.0      price 0.0000742500
Flash 2.0 Lite price 0.0000371250


In [27]:
prompt2 ="""
Summarize this artitle:
On 2 February 2025, Father Stephan Lipke, SJ, was ordained as the Auxiliary Bishop of the Diocese of the Transfiguration in Novosibirsk and titular Bishop of Arena. Fr Lipke is a member of the Province of Greater Poland and Mazovia which comprises parts of Poland, Denmark, Russia, Belarus, and Kyrgyzstan.
The episcopal consecration, which took place at the Cathedral of the Transfiguration of the Lord in Novosibirsk, was led by Bishop Joseph Werth, SJ, the ordinary of the Diocese of Transfiguration, and accompanied by other Bishops. Jesuits, government representatives, and a capacity crowd of faithful attended the ceremony.
Born on 31 December 1975 in Essen, Germany, Fr Lipke, pursued studies in philosophy and theology at the University of Bonn. He was ordained a deacon in 2001 and a priest in 2002. He joined the Society of Jesus in 2006 as a priest, and pronounced his First Vows in 2008. His mission to Russia began in 2011, where he held teaching and pastoral roles in Tomsk before earning a doctorate in Russian literature from Tomsk State University in 2017. He later underwent Tertianship in Manila, Philippines. In Moscow, he served as the director of the St. Thomas Institute, editor of the magazine Simvol, and Consultor to the Superior of the Independent Russian Region.
"""

In [35]:
tokens, chars =print_token(prompt2)
print_price(tokens, chars)

Prompt Token Count: 308
Prompt Character Count: 1101
Flash 1.5      price 0.0000206437
Flash 2.0      price 0.0000462000
Flash 2.0 Lite price 0.0000231000


In [47]:
prompt3="describe image"
tokens, chars =print_token(prompt3,"20250114134422 edit cat.png")
print_price(tokens, chars)

Prompt Token Count: 260
Prompt Character Count: 13
Flash 1.5      price 0.0000002437
Flash 2.0      price 0.0000390000
Flash 2.0 Lite price 0.0000195000


In [53]:
prompt3="describe image"
tokens, chars =print_token(prompt3)
print_price(tokens, chars)

Prompt Token Count: 2
Prompt Character Count: 13
Flash 1.5      price 0.0000002437
Flash 2.0      price 0.0000003000
Flash 2.0 Lite price 0.0000001500


In [68]:
prompt_en = """In the heart of a bustling city, amidst the towering skyscrapers and the ceaseless hum of traffic, lived a young girl named Lily. 
    Lily was a curious and imaginative child, with a heart full of dreams and a mind brimming with ideas."""
print_token(prompt_en)

Tokens count: 50
Tokens list: [TokensInfo(token_ids=[886, 573, 3760, 576, 476, 130898, 3413, 235269, 71557, 573, 121048, 171729, 578, 573, 179162, 2561, 576, 9988, 235269, 11012, 476, 3486, 4602, 8602, 41808, 235265, 235248, 108, 141, 68194, 729, 476, 17978, 578, 78674, 2047, 235269, 675, 476, 3760, 2247, 576, 15800, 578, 476, 3403, 225950, 675, 5793, 235265], tokens=[b'In', b' the', b' heart', b' of', b' a', b' bustling', b' city', b',', b' amidst', b' the', b' towering', b' skyscrapers', b' and', b' the', b' ceaseless', b' hum', b' of', b' traffic', b',', b' lived', b' a', b' young', b' girl', b' named', b' Lily', b'.', b' ', b'\n', b'    ', b'Lily', b' was', b' a', b' curious', b' and', b' imaginative', b' child', b',', b' with', b' a', b' heart', b' full', b' of', b' dreams', b' and', b' a', b' mind', b' brimming', b' with', b' ideas', b'.'], role='user')]


In [70]:
prompt_ko ="번잡한 도시의 중심부, 우뚝 솟은 고층 빌딩과 끊임없이 웅성거리는 교통 소음 속에서 릴리라는 어린 소녀가 살았습니다. 릴리는 호기심이 많고 상상력이 풍부한 아이였으며, 마음은 꿈으로 가득했고, 정신은 아이디어로 가득했습니다."

print_token(prompt_ko)

Tokens count: 89
Tokens list: [TokensInfo(token_ids=[238628, 241533, 236511, 204778, 236137, 47250, 239308, 237092, 235269, 54087, 247854, 235248, 248607, 236648, 46749, 242354, 235248, 242221, 241103, 237233, 235248, 245202, 238700, 238739, 235832, 235248, 242406, 237154, 238154, 137120, 75985, 238693, 44997, 238036, 125507, 22803, 235248, 241949, 236432, 139978, 53355, 239263, 192996, 236361, 130412, 239985, 21743, 235265, 235248, 241949, 137120, 83942, 236386, 239308, 235832, 99805, 236464, 36203, 237047, 238365, 235832, 209727, 237092, 236511, 72797, 239867, 101715, 235269, 41645, 238036, 236648, 235248, 242772, 26291, 23248, 242127, 238267, 236464, 235269, 35467, 237502, 236648, 72797, 175659, 236375, 23248, 242127, 76571, 235265], tokens=[b'\xeb\xb2\x88', b'\xec\x9e\xa1', b'\xed\x95\x9c', b' \xeb\x8f\x84\xec\x8b\x9c', b'\xec\x9d\x98', b' \xec\xa4\x91', b'\xec\x8b\xac', b'\xeb\xb6\x80', b',', b' \xec\x9a\xb0', b'\xeb\x9a\x9d', b' ', b'\xec\x86\x9f', b'\xec\x9d\x80', b' \xea\xb3\xa

In [37]:
import random
from PIL import Image

def create_random_image(width, height):
    output_path = f"./images/random/output_{width}_{height}.png"
    """
    Creates an image with random RGB pixel values and saves it as a PNG file.

    Args:
        width (int): The width of the image in pixels.
        height (int): The height of the image in pixels.
        output_path (str): The path to save the image.
    """

    # Create a new image with a white background
    img = Image.new('RGB', (width, height), color='white')

    # Generate random pixel values
    pixels = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
              for _ in range(width * height)]

    # Put the random pixels into the image
    img.putdata(pixels)

    # Save the image
    img.save(output_path)
    return output_path
